# Hidden Markov Model - Implementation of Viterby Algorithm
In this notebook we create a simple HMM and implement the Viterby algorithm to retrieve the most probable sequence out of a sequence of measurements.

In [1]:
# some imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from enum import IntEnum
%matplotlib inline

## HMM definition
We want to implement the following model.

In [2]:
# here a picture

Let's define some hidden and observable states

In [3]:
# hidden
class Weather(IntEnum):
    Sun = 0
    Cloud = 1
    Rain = 2

In [4]:
# observable
class Mood(IntEnum):
    Good = 0
    Bad = 1

### Transition matrix $A$
Now we define the state transition matrix $A$. Probability of changing from hidden state $i$ to state $j$ is saved in elements $a_{ij}$.

In [5]:
A = np.zeros(shape=(len(Weather), len(Weather)))

Let's create some probabilities $a_{ij}$

In [6]:
A[Weather.Sun, Weather.Sun] = 0.99
A[Weather.Sun, Weather.Cloud] = 0.01
A[Weather.Sun, Weather.Rain] = 0
A[Weather.Cloud, Weather.Sun] = 0.0
A[Weather.Cloud, Weather.Cloud] = 0.0
A[Weather.Cloud, Weather.Rain] = 1
A[Weather.Rain, Weather.Sun] = 0.0
A[Weather.Rain, Weather.Cloud] = 0.0
A[Weather.Rain, Weather.Rain] = 1.0

In [7]:
A

array([[ 0.99,  0.01,  0.  ],
       [ 0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  ,  1.  ]])

### Observation matrix $B$
Now we define the probabilities $b_{ij}$ observing curtain value j given a hidden state i. We save them in the matrix $B$.

In [8]:
B = np.zeros(shape=(len(Weather), len(Mood)))

Let's define some probabilities:

In [9]:
B[Weather.Sun, Mood.Good] = 0.99
B[Weather.Sun, Mood.Bad] = 0.01
B[Weather.Cloud, Mood.Good] = 0.01
B[Weather.Cloud, Mood.Bad] = 0.99
B[Weather.Rain, Mood.Good] = 0
B[Weather.Rain, Mood.Bad] = 1

In [10]:
B

array([[ 0.99,  0.01],
       [ 0.01,  0.99],
       [ 0.  ,  1.  ]])

### Initialization probability $\pi$
How likely is that the hidden state $i$ is the first state? We express this probability in values $\pi_i$.

In [11]:
Pi = np.zeros(len(Weather))

In [12]:
Pi[:] = [1.0, 0.0, 0.0]

In [13]:
Pi

array([ 1.,  0.,  0.])

## Problem
Let's define an observed sequence. We want to determine the most probable underlying states

In [14]:
sequence = [Mood.Good, Mood.Good, Mood.Bad, Mood.Good, Mood.Bad, Mood.Bad]

In [15]:
sequence = [Mood.Good, Mood.Good, Mood.Good]

## Algorithm
Here some wikipeda quotes ...

In order to run the algorithm we have to allocate some memory to save the variables $\vartheta_{t}(i)$

In [16]:
T = np.zeros(shape=(len(sequence), len(Weather)))

The same with the variable $\varphi_{t}(i)$

In [17]:
P = np.zeros(shape=(len(sequence), len(Weather)))

### Initialization

In [18]:
# first observation
o1 = sequence[0]

In [19]:
T[0, :] = Pi[o1]*B[:,o1]

In [20]:
T

array([[ 0.99,  0.01,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ]])

### Recursion
Let's define the recursion step here:

In [21]:
def step(t, s):
    if t > 0:
        for i in range(len(Weather)):
            a_v_max = 0
            j_max = 0
            for j in range(len(Weather)):
                v_prev_j, _ = step(t-1, s)
                a_ji = A[j,i]
                a_v = a_ji*v_prev_j
                if a_v > a_v_max:
                    j_max = j
                    a_v_max = a_v
            P[t, i] = j
            T[t, s] = B[s, sequence[t]]*a_v_max
        return T[t, s], P[t]
    else:
        return T[t, s], P[t]

In [22]:
for i in range(len(Weather)):
    T[len(sequence)-1, i], _ = step(len(sequence)-1, i)

In [23]:
T

array([[  9.90000000e-01,   1.00000000e-02,   0.00000000e+00],
       [  9.80100000e-01,   1.00000000e-04,   0.00000000e+00],
       [  9.70299000e-01,   1.00000000e-06,   0.00000000e+00]])